In [1]:
from decimal import Decimal

from btopt.data.bar import Bar
from btopt.data.dataloader import CSVDataLoader
from btopt.engine import Engine
from btopt.log_config import logger_main
from btopt.order import Order
from btopt.strategy.strategy import Strategy

In [2]:
class Empty(Strategy):
    def __init__(
        self,
    ) -> None:
        super().__init__()

    def on_data(self, bar: Bar) -> None:
        x = Engine()
        x._dataview.data

        if len(self) <= 1:
            return

        self.debug(f"PRIMARY: {(self.data.close[1])}, TIMEFRAME: {self.datas.keys()}")
        self.debug(f"TIMEFRAMES: {self.datas[self._primary_symbol].timeframes}")
        # self.debug(
        #     f"SECONDARY: {(self.datas[self._primary_symbol].get(value="close"))}, TIMEFRAME: {self.datas[self._primary_symbol][1]._timeframe}"
        # )

        # if len(self) > 100:
        #     raise

    @staticmethod
    def debug(message):
        logger_main.info(message)


class SimpleMovingAverageCrossover(Strategy):
    def __init__(self, fast_period: int = 10, slow_period: int = 20):
        # super().__init__()
        self.fast_period = fast_period
        self.slow_period = slow_period
        self.fast_ma = []
        self.slow_ma = []

        self.warmup_period = self.slow_period

    def on_data(self) -> None:
        bar = self.datas[self._primary_symbol].get(index=0)
        close_prices = self.datas[self._primary_symbol].get(
            size=self.slow_period + 1, value="close"
        )

        self.fast_ma.append(sum(close_prices[-self.fast_period :]) / self.fast_period)
        self.slow_ma.append(sum(close_prices[-self.slow_period :]) / self.slow_period)

        if len(self.fast_ma) > 1 and len(self.slow_ma) > 1:
            current_position = self.get_current_position(bar.ticker)

            if (
                self.fast_ma[-1] > self.slow_ma[-1]
                and self.fast_ma[-2] <= self.slow_ma[-2]
            ):
                # Bullish crossover
                if current_position <= 0:
                    size = (
                        abs(current_position) + 1
                    )  # Close short (if any) and open long
                    self.buy(bar.ticker, size)
                    logger_main.info(f"Bullish crossover: Buying {size} {bar.ticker}")

            elif (
                self.fast_ma[-1] < self.slow_ma[-1]
                and self.fast_ma[-2] >= self.slow_ma[-2]
            ):
                # Bearish crossover
                if current_position >= 0:
                    size = current_position + 1  # Close long (if any) and open short
                    self.sell(bar.ticker, size)
                    logger_main.info(f"Bearish crossover: Selling {size} {bar.ticker}")

    def on_order_update(self, order: Order) -> None:
        logger_main.info(f"Order update: {order}")

    def on_trade_update(self, trade) -> None:
        logger_main.info(f"Trade update: {trade}")

In [3]:
def run_backtest():
    # Initialize the engine
    engine = Engine()

    # Load data
    symbol = "EURUSD"
    start_date = "2022-01-01"
    end_date = "2023-01-01"

    ctf, htf = "1h", "1d"

    dataloader = CSVDataLoader(symbol, "1m", start_date=start_date, end_date=end_date)

    # engine.resample_data(dataloader, ctf)
    engine.resample_data(dataloader, htf)

    # # Create and add the strategy
    # strategy = SimpleMovingAverageCrossover(
    #     "SMA Crossover", fast_period=10, slow_period=20
    # )

    # engine.add_strategy(Empty, ctf, htf)
    engine.add_strategy(SimpleMovingAverageCrossover, fast_period=10, slow_period=20)

    # Set up the backtest configuration

    initial_capital = Decimal("100000")
    commission_rate = Decimal("0.001")  # 0.1% commission
    config = {
        "initial_capital": initial_capital,
        "commission_rate": commission_rate,
    }
    engine.set_config(config)

    # Run the backtest
    try:
        logger_main.info("Starting backtest")
        reporter = engine.run()

        return reporter
    except Exception as e:
        logger_main.error(f"Error during backtest: {e}", exc_info=True)

In [4]:
reporter = run_backtest()

Log file '/Users/jerryinyang/Code/btopt/logs/main.log' has been cleared successfully.


/Users/jerryinyang/Code/btopt/btopt/portfolio.py:231: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.metrics = pd.concat([self.metrics, new_row], ignore_index=True)


In [5]:
reporter.calculate_calmar_ratio()

AttributeError: 'NoneType' object has no attribute 'calculate_calmar_ratio'

In [ ]:
if reporter:
    print("Backtest completed successfully.")
    print("Performance Summary:")
    print(reporter.generate_performance_summary())
else:
    print("Backtest failed.")

In [ ]:
reporter